Extracts car wise features. Does not write to h5

In [1]:
using AutomotiveDrivingModels
using AutoRisk
using HDF5
using NGSIM

INFO: Recompiling stale cache file /home/raunak/.julia/lib/v0.6/AutoRisk.ji for module AutoRisk.


## Caution: The below `build_feature_extractor` needs `AutoRisk` to be on laneid branch if you are including the laneidfeatureextractor

In [2]:
function build_feature_extractor()
    subexts = [
        LaneIDFeatureExtractor(),
        CoreFeatureExtractor(),
        TemporalFeatureExtractor(),
        WellBehavedFeatureExtractor(),
        CarLidarFeatureExtractor(20, carlidar_max_range = 50.),
        ForeForeFeatureExtractor()
    ]
    ext = MultiFeatureExtractor(subexts)
    return ext
end

build_feature_extractor (generic function with 1 method)

In [9]:
function extract_features(
        ext,
        trajdata, 
        roadway, 
        timestep_delta, 
        record_length, 
        offset, 
        prime,
        maxframes)
    n_features = length(ext)
    max_n_objects = maximum(n_objects_in_frame(trajdata, i) for i in 1 : nframes(trajdata))
    scene = Scene(max_n_objects)
    rec = SceneRecord(record_length, 0.1, max_n_objects)
    features = Dict{Int, Array{Float64}}()
    ctr = 0
    n_frames = nframes(trajdata)
    
    for frame in (offset - prime : offset - 1)
        # prime the rec
        AutomotiveDrivingModels.update!(rec, get!(scene, trajdata, frame))
    end

    veh_features = pull_features!(ext, rec, roadway, 1)
#     println("veh_features shape = $(size(veh_features))")

    for frame in offset : (n_frames - offset)
#     for frame in offset : offset+1
        ctr += 1
#         println("ctr = $(ctr)")
        if maxframes != nothing && ctr >= maxframes
            break
        end

        print("\rframe $(frame) / $(n_frames - offset)")
            
        # update the rec
        AutomotiveDrivingModels.update!(rec, get!(scene, trajdata, frame))

        # every timestep_delta step, extract features
        if frame % timestep_delta == 0
            for (vidx, veh) in enumerate(scene)
                # extract features
                veh_features = pull_features!(ext, rec, roadway, vidx)
                #println("veh.id = $(veh.id) \n")
                
                # add entry to features if vehicle not yet encountered
                if !in(veh.id, keys(features))
                    features[veh.id] = zeros(n_features, 0)
                end

                # stack onto existing features
                features[veh.id] = cat(2, features[veh.id], 
                    reshape(veh_features, (n_features, 1)))
            end
        end
    end
    
    
#     println("shape features of vehicle 23= $(size(features[23]))")
#     println("length of keys of features i.e total number 
#         of distinct vehicles encountered \n $(length(keys(features)))")
    
    # Print the shape of the associated feature for every vehicle index
#     for (key,value) in features
#         println("key: $(key) ==> shape: $(size(value))")
#     end
        
    return features
end

extract_features (generic function with 1 method)

In [10]:
        timestep_delta = 1 # timesteps between feature extractions
        record_length = 10 # number of frames for record to track in the past
        offset = 500 # from ends of the trajectories
        prime = 10
        maxframes = nothing # nothing for no max

    ext = build_feature_extractor()
    features = Dict{Int, Dict{Int, Array{Float64}}}()

    tic()
    # extract 


        # setup
        trajdata = load_trajdata(1)
        roadway = get_corresponding_roadway(1)
        features = extract_features(
            ext, 
            trajdata, 
            roadway, 
            timestep_delta, 
            record_length, 
            offset, 
            prime,
            maxframes
        )
    toc()


frame 9036 / 9036elapsed time: 807.055164695 seconds


807.055164695

In [11]:
# This features thing is gold. It is the ngsim data extracted carwise above
length(keys(features))

2150

In [36]:
function calcLaneChanges(test::Array{Float64})
lanechanges = 0
for i = 1:length(test)-1
    if test[i] != test[i+1]
        lanechanges = lanechanges + 1
    end
end
return lanechanges
end

calcLaneChanges (generic function with 1 method)

In [38]:
calcLaneChanges(features[233][1,:])

4

In [39]:
laneChangeData = Array{Float64}
for (key,value) in features
#    if verbose
#         println("size of features = $(size(value))")
#     end
    laneData = value[1,:] # Note that 1 is the index of lane id 
    numLaneChanges = calcLaneChanges(laneData)
    timeHorizon = length(laneData)
    
    println("$(numLaneChanges) $(timeHorizon)")
end

3 271
2 442
2 840
2 401
2 517
3 656
4 323
2 515
5 812
2 431
2 592
3 697
2 741
4 556
3 582
3 473
2 463
2 422
2 673
2 649
2 471
2 428
3 470
2 598
2 679
2 466
2 530
2 543
2 676
2 451
2 581
9 359
2 564
2 451
3 474
2 856
2 711
2 960
2 453
2 574
2 345
4 706
2 495
2 510
3 430
2 509
2 683
2 643
2 458
2 507
5 509
4 968
2 776
2 690
2 526
2 919
2 435
2 507
2 760
1 204
2 468
2 504
3 553
2 674
2 290
2 474
2 469
2 554
2 447
2 526
2 172
2 678
3 441
2 427
2 677
2 742
2 965
2 499
2 453
11 499
1 431
2 468
2 562
2 804
1 125
3 463
2 516
0 137
2 601
2 399
2 453
5 565
2 553
2 511
2 430
2 470
2 451
2 424
2 457
2 660
1 310
4 460
2 742
2 639
2 370
2 782
2 305
3 850
2 519
2 608
2 724
2 470
2 489
3 546
2 598
2 924
3 549
2 565
2 424
5 471
2 524
2 948
2 520
2 584
2 472
2 507
10 655
3 351
3 380
2 426
2 653
2 508
2 671
3 375
2 937
3 544
2 936
2 445
2 551
2 509
2 590
1 211
2 508
0 4
2 442
2 487
0 47
2 471
2 477
4 666
2 550
2 720
2 575
5 503
2 491
2 488
2 713
2 567
4 514
3 820
2 466
2 286
2 500
2 378
2 470
2 417
2 856

2 550
2 510
2 555
2 581
3 846
2 770
2 459
2 512
2 426
2 447
2 380
2 541
2 944
2 514
2 492
2 443
2 591
2 610
2 476
2 685
2 754
2 530
3 320
3 495
1 574
2 690
2 502
2 339
2 503
3 423
2 927
2 605
2 422
3 961
2 471
2 517
3 835
2 292
2 514
2 562
1 256
2 385
3 462
2 663
2 255
3 485
2 422
2 554
2 434
2 433
2 664
2 448
2 295
2 454
2 464
2 455
2 506
2 500
1 535
2 629
5 385
2 516
2 591
2 489
2 556
3 482
1 672
3 547
2 476
1 456
4 520
2 486
2 445
2 453
2 825
0 78
2 559
2 457
2 496
2 680
2 683
1 245
2 712
4 388
2 821
2 447
2 821
2 565
2 565
2 945
3 409
2 450
2 552
0 71
2 439
2 611
2 805
2 744
1 497
2 559
2 240
2 802
2 902
2 570
2 513
2 644
2 721
2 429
4 443
1 425
1 649
2 710
2 289
0 306
2 951
2 479
0 274
2 592
2 756
0 99
2 409
2 335
2 421
3 431
2 642
2 501
3 462
2 650
2 650
3 504
2 427
2 528
2 506
2 939
2 482
2 466
2 435
2 552
2 425
2 489
2 564
2 514
2 504
2 580
2 720
2 452
2 532
2 601
2 932
2 512
6 478
3 378
2 501
2 505
2 347
1 350
4 533
2 276
2 542
2 545
3 850
3 517
2 468
2 483
2 469
5 545
2 433
1

In [ ]:
println(feature)

In [41]:
# print the timegaps
# timegap will be 16 index. Recall laneid was index 1
for (key,value) in features
    println(size(value))
end

(67, 271)
(67, 442)
(67, 840)
(67, 401)
(67, 517)
(67, 656)
(67, 323)
(67, 515)
(67, 812)
(67, 431)
(67, 592)
(67, 697)
(67, 741)
(67, 556)
(67, 582)
(67, 473)
(67, 463)
(67, 422)
(67, 673)
(67, 649)
(67, 471)
(67, 428)
(67, 470)
(67, 598)
(67, 679)
(67, 466)
(67, 530)
(67, 543)
(67, 676)
(67, 451)
(67, 581)
(67, 359)
(67, 564)
(67, 451)
(67, 474)
(67, 856)
(67, 711)
(67, 960)
(67, 453)
(67, 574)
(67, 345)
(67, 706)
(67, 495)
(67, 510)
(67, 430)
(67, 509)
(67, 683)
(67, 643)
(67, 458)
(67, 507)
(67, 509)
(67, 968)
(67, 776)
(67, 690)
(67, 526)
(67, 919)
(67, 435)
(67, 507)
(67, 760)
(67, 204)
(67, 468)
(67, 504)
(67, 553)
(67, 674)
(67, 290)
(67, 474)
(67, 469)
(67, 554)
(67, 447)
(67, 526)
(67, 172)
(67, 678)
(67, 441)
(67, 427)
(67, 677)
(67, 742)
(67, 965)
(67, 499)
(67, 453)
(67, 499)
(67, 431)
(67, 468)
(67, 562)
(67, 804)
(67, 125)
(67, 463)
(67, 516)
(67, 137)
(67, 601)
(67, 399)
(67, 453)
(67, 565)
(67, 553)
(67, 511)
(67, 430)
(67, 470)
(67, 451)
(67, 424)
(67, 457)
(67, 660)


(67, 463)
(67, 583)
(67, 500)
(67, 477)
(67, 450)
(67, 760)
(67, 656)
(67, 490)
(67, 557)
(67, 297)
(67, 480)
(67, 420)
(67, 512)
(67, 631)
(67, 797)
(67, 629)
(67, 609)
(67, 559)
(67, 555)
(67, 873)
(67, 327)
(67, 536)
(67, 3)
(67, 653)
(67, 628)
(67, 719)
(67, 488)
(67, 439)
(67, 541)
(67, 490)
(67, 530)
(67, 768)
(67, 586)
(67, 506)
(67, 828)
(67, 545)
(67, 487)
(67, 672)
(67, 423)
(67, 509)
(67, 526)
(67, 405)
(67, 470)
(67, 405)
(67, 509)
(67, 479)
(67, 664)
(67, 74)
(67, 540)
(67, 524)
(67, 744)
(67, 514)
(67, 421)
(67, 606)
(67, 481)
(67, 423)
(67, 933)
(67, 562)
(67, 431)
(67, 579)
(67, 442)
(67, 560)
(67, 460)
(67, 426)
(67, 423)
(67, 796)
(67, 226)
(67, 331)
(67, 834)
(67, 134)
(67, 471)
(67, 424)
(67, 683)
(67, 458)
(67, 603)
(67, 341)
(67, 795)
(67, 935)
(67, 50)
(67, 659)
(67, 448)
(67, 765)
(67, 720)
(67, 462)
(67, 883)
(67, 378)
(67, 549)
(67, 490)
(67, 577)
(67, 459)
(67, 511)
(67, 730)
(67, 670)
(67, 527)
(67, 402)
(67, 287)
(67, 669)
(67, 513)
(67, 759)
(67, 446)
(67,

(67, 446)
(67, 480)
(67, 239)
(67, 174)
(67, 683)
(67, 454)
(67, 430)
(67, 606)
(67, 504)
(67, 534)
(67, 544)
(67, 644)
(67, 828)
(67, 525)
(67, 306)
(67, 316)
(67, 428)
(67, 594)
(67, 534)
(67, 327)
(67, 601)
(67, 631)
(67, 465)
(67, 676)
(67, 320)
(67, 416)
(67, 528)
(67, 500)
(67, 464)
(67, 543)
(67, 308)
(67, 439)
(67, 464)
(67, 305)
(67, 511)
(67, 721)
(67, 438)
(67, 465)
(67, 2)
(67, 835)
(67, 578)
(67, 501)
(67, 779)
(67, 540)
(67, 283)
(67, 466)
(67, 363)
(67, 263)
(67, 426)
(67, 284)
(67, 352)
(67, 474)
(67, 448)
(67, 548)
(67, 548)
(67, 514)
(67, 526)
(67, 489)
(67, 687)
(67, 631)
(67, 425)
(67, 501)
(67, 320)
(67, 463)
(67, 448)
(67, 612)
(67, 515)
(67, 655)
(67, 947)
(67, 461)
(67, 510)
(67, 619)
(67, 427)
(67, 511)
(67, 480)
(67, 541)
(67, 562)
(67, 583)
(67, 573)
(67, 772)
(67, 543)
(67, 429)
(67, 512)
(67, 528)
(67, 250)
(67, 575)
(67, 538)
(67, 468)
(67, 700)
(67, 502)
(67, 440)
(67, 648)
(67, 565)
(67, 738)
(67, 653)
(67, 716)
(67, 503)
(67, 526)
(67, 725)
